In [18]:
import yfinance as yf
import pandas as pd
import openpyxl
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import optimize

import os
cwd = os.getcwd()
print(cwd)
os.chdir(r'C:\Users\mitch\Documents\QCF\Finance and Investments\StockTrack-Trading-Simulation')

C:\Users\mitch\Documents\QCF\Finance and Investments\StockTrack-Trading-Simulation


# Reusable functions

In [19]:
def calculate_return(df):
    if df.empty:
        return None
    # Get the closing price at the start of the year
    start_price = df['Close'].iloc[0]
    
    # Get the closing price at the end of the year
    end_price = df['Close'].iloc[-1]
    
    # Calculate the one-year return
    return (end_price - start_price) / start_price

def MaximizeSharpeRatioOptmzn(MeanReturns, CovarReturns, RiskFreeRate, PortfolioSize):
     # define maximization of Sharpe Ratio using principle of duality
    def  f(x, MeanReturns, CovarReturns, RiskFreeRate, PortfolioSize):
        funcDenomr = np.sqrt(np.matmul(np.matmul(x, CovarReturns), x.T) )
        funcNumer = np.matmul(np.array(MeanReturns),x.T)-RiskFreeRate
        func = -(funcNumer / funcDenomr)
        return func

    #define equality constraint representing fully invested portfolio
    def constraintEq(x):
        A=np.ones(x.shape)
        b=1
        constraintVal = np.matmul(A,x.T)-b 
        return constraintVal
    
    #define bounds and other parameters
    xinit=np.repeat(0.33, PortfolioSize)
    cons = ({'type': 'eq', 'fun':constraintEq})
    lb = 0
    ub = 1
    bnds = tuple([(lb,ub) for x in xinit])
    
    #invoke minimize solver
    opt = optimize.minimize (f, x0 = xinit, args = (MeanReturns, CovarReturns,\
                             RiskFreeRate, PortfolioSize), method = 'SLSQP',  \
                             bounds = bnds, constraints = cons, tol = 10**-3)
    
    return opt
    

In [20]:
largest_market_cap = pd.read_excel('./Top Companies by Market Cap.xlsx', sheet_name='Sheet1')
largest_market_cap

,Symbol,Company Name,Industry,Market Cap
0,AAPL,Apple Inc.,Consumer Electronics,"2,811.03B"
1,MSFT,Microsoft Corporation,Software - Infrastructure,"2,469.80B"
2,GOOGL,Alphabet Inc.,Internet Content & Information,"1,779.01B"
3,GOOG,Alphabet Inc.,Internet Content & Information,"1,778.79B"
4,AMZN,"Amazon.com, Inc.",Internet Retail,"1,360.20B"
...,...,...,...,...
495,RBLX,Roblox Corporation,Electronic Gaming & Multimedia,19.59B
496,WBA,"Walgreens Boots Alliance, Inc.",Pharmaceutical Retailers,19.51B
497,TW,Tradeweb Markets Inc.,Capital Markets,19.51B
498,ZM,"Zoom Video Communications, Inc.",Software - Application,19.51B


In [21]:
largest_market_cap = largest_market_cap[~largest_market_cap['Symbol'].str.contains('\.')].reset_index(drop=True)
top_300 = largest_market_cap.head(100)
top_300_symbols = top_300['Symbol']

In [22]:
# ticker = yf.Ticker('AAPL')
# stock_info = ticker.info
# print(stock_info.get('beta'))

In [23]:
one_year_data = {}
three_year_data = {}
five_year_data = {} 
for symbol in top_300_symbols:
    ticker = yf.Ticker(symbol)
    one_year_data[symbol] = ticker.history(period="1y")
    three_year_data[symbol] = ticker.history(period="3y")
    five_year_data[symbol] = ticker.history(period="5y")

In [24]:
one_year_return_data = {}
three_year_return_data = {}
five_year_return_data = {}
for key, df in one_year_data.items():
    one_year_return_data[key] = calculate_return(one_year_data[key])
    three_year_return_data[key] = calculate_return(three_year_data[key])
    five_year_return_data[key] = calculate_return(five_year_data[key])
    
one_year_return_data

{'AAPL': 0.2532569689154102,
 'MSFT': 0.53269479182366,
 'GOOGL': 0.38025078944704027,
 'GOOG': 0.3916484545291894,
 'AMZN': 0.4477611882489815,
 'NVDA': 1.985562512022651,
 'META': 1.8820696806448503,
 'TSLA': 0.17156329033356696,
 'LLY': 0.7345737675515458,
 'V': 0.19217314239362615,
 'UNH': 0.07164587129098937,
 'TSM': 0.3514773473407748,
 'NVO': 0.8071784164159003,
 'WMT': 0.2301976105292276,
 'JPM': 0.12109221626815235,
 'XOM': -0.052332967298438345,
 'JNJ': -0.1165411800264168,
 'MA': 0.16909607459887038,
 'AVGO': 0.8941051878114836,
 'PG': 0.10888360395586448,
 'CVX': -0.1999408593435794,
 'ORCL': 0.4756670123596331,
 'HD': -0.03593613990157222,
 'ABBV': -0.04942156536880995,
 'MRK': 0.04905661084746388,
 'COST': 0.14308729081994384,
 'ADBE': 0.7344067788707931,
 'TM': 0.3209964251572046,
 'ASML': 0.14417260642599244,
 'KO': -0.03383187323422096,
 'PEP': -0.024791635770954744,
 'BABA': 0.15323145975887562,
 'SHEL': 0.2442411052018249,
 'CSCO': 0.20319611329451912,
 'BAC': -0.245

In [25]:
def get_deciles(data):
    # Remove None values and sort the data
    filtered_data = {k: v for k, v in data.items() if v is not None}
    sorted_data = sorted(filtered_data.items(), key=lambda x: x[1])

    # Calculate the index for decile
    n = len(sorted_data)
    decile_index = n // 10

    # Get the top and bottom decile
    bottom_decile = dict(sorted_data[:decile_index])
    top_decile = dict(sorted_data[-decile_index:])

    return top_decile, bottom_decile

# Apply the function to each dataset
top_decile_one_year, bottom_decile_one_year = get_deciles(one_year_return_data)
top_decile_three_year, bottom_decile_three_year = get_deciles(three_year_return_data)
top_decile_five_year, bottom_decile_five_year = get_deciles(five_year_return_data)

# Find overlapping keys in the top deciles
overlapping_top_keys = set(top_decile_one_year.keys()) & set(top_decile_three_year.keys()) & set(top_decile_five_year.keys())

# Find overlapping keys in the bottom deciles
overlapping_bottom_keys = set(bottom_decile_one_year.keys()) & set(bottom_decile_three_year.keys()) & set(bottom_decile_five_year.keys())

# Print results
print("Top Decile Overlaps:", overlapping_top_keys)
print("Bottom Decile Overlaps:", overlapping_bottom_keys)


Top Decile Overlaps: {'LLY', 'NVDA', 'NVO'}
Bottom Decile Overlaps: set()


In [ ]:
'''''
filtered_data_one_year = {key: value for key, value in one_year_return_data.items() if value is not None and value >= 0.15 and value <= 0.8}
filtered_data_three_year = {key: value for key, value in three_year_return_data.items() if value is not None and value >= 0.10}
filtered_data_five_year = {key: value for key, value in five_year_return_data.items() if value is not None and value >= 0.10}

print(filtered_data_one_year.values())
print(filtered_data_three_year.values())
print(filtered_data_five_year.values())
overlapping_keys = set(filtered_data_one_year.keys()) & set(filtered_data_three_year.keys()) & set(filtered_data_five_year.keys())
'''


In [ ]:
overlapping_keys

In [ ]:
betas = {}

for key in overlapping_keys:
    try:
        beta = yf.Ticker(key).info.get('beta')
        if beta is not None and 0.70 < beta <= 2.50:
            betas[key] = beta
    except Exception as e:
        print(f"An error occurred for ticker {key}: {e}")

print(len(betas))

In [ ]:
filtered_beta = {key: value for key, value in betas.items() if value is not None and value > 0.70 and value <= 2.50}

df_filtered_beta_keys = pd.DataFrame(list(filtered_beta.keys()), columns=['Symbol'])
df_filtered_beta_keys

In [ ]:
merged_df = pd.merge(df_filtered_beta_keys, largest_market_cap, on='Symbol', how='left')
merged_df

### Semiconductor Equipment & Materials: ASML Holding N.V. (ASML)
### Internet Content & Information: Alphabet Inc. (GOOG or GOOGL)
### Information Technology Services: Accenture plc (ACN)
### Consumer Electronics: Apple Inc. (AAPL)
### Software - Application: Intuit Inc. (INTU)
### Credit Services: Visa Inc. (V)
### Asset Management: Blackstone Inc. (BX)
### Oil & Gas Integrated: TotalEnergies SE (TTE)
### Travel Services: Booking Holdings Inc. (BKNG)
### Software - Infrastructure: Microsoft Corporation (MSFT)

# Pull past 1 years' data and calculate their daily actual returns

In [ ]:
stocks = ["ASML", "GOOG", "ACN", "AAPL", "INTU", "V", "BX", "TTE", "BKNG", "MSFT"]

In [ ]:
daily_actual_returns = {}
for s in stocks:
    daily_actual_returns[s] = one_year_data[s]['Close'].pct_change(1).dropna()

In [ ]:
combined_daily_actual_returns = pd.concat(daily_actual_returns, axis=1)
combined_daily_actual_returns

In [ ]:
five_year_expected_returns = combined_daily_actual_returns.mean().to_frame(name='Annual Expected Return')*252
five_year_expected_returns['Annual Expected Return'].to_frame().T

In [ ]:
five_year_std_dev = np.sqrt(combined_daily_actual_returns.var().to_frame(name='Standard Deviation')*252)
five_year_std_dev['Standard Deviation'].to_frame().T

In [ ]:
cov_matrix = combined_daily_actual_returns.cov()
display(cov_matrix)
plt.figure(figsize=(10, 8))
sns.heatmap(cov_matrix, annot=True, cmap='coolwarm', fmt='.4f')
plt.title("Covariance Matrix")
plt.show()

### Get the 30 years Treasury Bond

In [ ]:
tyx = yf.Ticker("^TYX")

# Get the historical data (most recent data)
data = tyx.history(period="5d")

# Extract the close value, which represents the yield
latest_yield = data['Close'].iloc[0]

print(f"Current 30-year Treasury bond yield: {latest_yield}%")